<a href="https://colab.research.google.com/github/fblondiaux/LGBIO2072-2020/blob/update/LGBIO2072_TP1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Leaky Integrate and Fire (LIF) Tutorial
__Authors:__ Antoine de Comite and Florence Blondiaux

__Content inspired from__: Neuromatch Academy github.com/NeuromatchAcademy

Please run the cells below to initialize the notebook.

In [ ]:
#Useful libraries
import numpy as np
import matplotlib.pyplot as plt



In [ ]:
# @title Figure Settings
import ipywidgets as widgets  # interactive display
%config InlineBackend.figure_format = 'retina'
my_layout = widgets.Layout()

plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/nma.mplstyle")

---
## Tutorial Objectives
 In this tutorial, we will build up a leaky integrate-and-fire (LIF) neuron model and study its dynamics in response to various types of inputs. In particular, we are going to write a few lines of code to:  

- simulate the LIF neuron model

- drive the LIF neuron with external inputs, such as direct currents, Gaussian white noise, sinusoidal currents, ...
- study how different inputs affect the  LIF neuron's output (firing rate and spike time irregularity)

Here, we will especially emphasize identifying conditions (input statistics) under which a neuron can spike at low firing rates and in an irregular manner. The reason for focusing on this is that in most cases, neocortical neurons spike in an irregular manner.


---
 <font color='red'> If you are working in google colab, in order to save your work, go to File > "Save a copy in Drive" and work on the copy. If you don't do this, you will loose all your modifications when you close the tab ! </font>


---
## Reminder

A *membrane equation* and a *reset condition* define our *leaky-integrate-and-fire (LIF)* neuron:


\begin{equation*}
\tau_m \frac{du}{dt} = -\left(u(t)-u_{rest}\right) +RI(t)  \;\;\; if \; u(t)\leq \theta
\end{equation*}
\begin{equation*}
u(t) = u_{reset}  \;\;\;\;\;\;\;\;\;  otherwise
\end{equation*}


The membrane equation is an *ordinary differential equation (ODE)* that describes the time evolution of membrane potential $u(t)$ in response to synaptic input and leaking of change across the cell membrane. To find an approximation of the solution of this ODE, you can use explicit Euler integration over one time step.
Euler integration :
\begin{equation*}
\frac{dX(t)}{dt} \simeq \frac{X(t+\Delta t) - X(t)}{\Delta t}
\end{equation*}

## Synaptic input

We start with a sinusoidal model to simulate the synaptic input $I(t)$ given by:
\begin{align*}
\\
I(t)=I_{mean}\left(1+\sin\left(\frac{2 \pi}{0.01}\,t\right)\right)\\
\\
\end{align*}

### Exercise 1
Compute the values of synaptic input $I(t)$ 

In [ ]:
def syn_in(t, imean = 25e-11):
    """
    Function that produces sinusoidal input

    Args:
      t          : Vector of discretized time points (numpy array)
      imean      : current mean amplitude [A] (integer)

    Returns:
      sin_i       : Sinusoidal input current (numpy array)
    """
    sin_i = imean * (1 + np.sin((t * 2 * np.pi) / 0.01))
    return sin_i

t_max = 150e-3   # second
dt = .1e-3        # second
t_range = np.arange(0,t_max, dt)

with plt.xkcd():
    plt.plot(t_range,syn_in(t_range))
    plt.title("Synaptic Input I(t)")
    plt.xlabel('T [s]')
    plt.ylabel('I(t) [A]')
    plt.show()


*Example output:*

<img alt='Solution hint' align='left' width=413 height=300 src=https://github.com/mpbrigham/colaboratory-figures/raw/master/nma/python-for-nma/synaptic_input.png>

---
## ODE integration without spikes
In the next exercise, we now simulate the evolution of the membrane equation in discrete time steps, with a sufficiently small $\Delta t$.

**Start by writing an approximation for the time derivative $\dfrac{d}{dt}\,u(t)$ in discrete time** using discrete-time integration methods such as Euler method. See [Numerical methods for ordinary differential equations](https://en.wikipedia.org/wiki/Numerical_methods_for_ordinary_differential_equations) for additional details.

### Exercise 2 
Implement the evolution of the membrane potential in response to an input_current. Use the synusoidal input you just coded but try also with a step current as an input.

In [ ]:
def membranePot(input_current, t, tau= 20e-3 , urest= -60e-3 , r= 100e6):
    """
    Authors : Antoine de Comite and Florence Blondiaux
    Simulate the integration of the membrane potential with external input current

    Args:
      input_current   : Vector containing the input current [A].
      t               : Vector containing the discretized time points [s].
      tau             : Membrane time constant. Default value: 20e-3 [s]
      urest           : Leak potential. Default value: -60e-3 [V]
      r               : Leak resistance. Default value: 100e6 [Ohm]
    Returns:
      u               : membrane potential [V]
    """
    dt = t[1]-t[0]
    u = np.zeros(len(input_current))
    u[0]=urest
    for step in range(len(input_current)-1):
        u[step+1]=u[step] + dt/tau * ( -u[step] + urest +r* input_current[step+1])
    return u

#Generates a random input current
t_max = 150e-3   # second
dt = .1e-3        # second
t_range = np.arange(0,t_max,dt)
input_current = syn_in(t_range)
u = membranePot(input_current, t_range)

with plt.xkcd():
    fig, axs = plt.subplots(2)
    axs[0].plot(t_range,input_current)
    axs[0].set(title = "Input current", ylabel = "I [A]")
    axs[1].plot(t_range,u)
    axs[1].set(title = "Membrane potential", ylabel = "u [V]", xlabel= "Time [S]")
    plt.show()

*Example output:*


<img alt='Solution hint' align='left' width=413 height=300 src=https://raw.githubusercontent.com/fblondiaux/LGBIO2072-2020/master/Images/TP1-Ex2.png>

---
# Introduce spikes
A spike takes place whenever $u(t)$ crosses $\theta$. In that case, a spike is recorded and $u(t)$ resets to $u_{reset}$ value. This is summarized in the *reset condition*:
$$u(t) = u_{reset}\quad \text{ if } u(t)\geq \theta$$

For more information about spikes or action potentials see [here](https://en.wikipedia.org/wiki/Action_potential).


![spikes cartoon](https://raw.githubusercontent.com/fblondiaux/LGBIO2072-2020/master/Solutions/spikes_carton.png)

### Exercise 3 
Add the spiking process to your LIF model

In [ ]:
def simple_LIF(input_current,t,  tau= 20e-3 , urest= -60e-3 , r= 100e6, ureset = -70e-3 ,theta = -50e-3 ):
    """
    Authors : Antoine de Comite and Florence Blondiaux
    Simulate the integration of the membrane potential and the generation of spikes
    in response to an external input current

    Args:
      input_current   : Vector containing the input current [A].
      t               : Vector containing the discretized time points [s]
      tau             : Membrane time constant. Default value: 20e-3 [s]
      urest           : Leak potential. Default value: -60e-3 [V]
      r               : Leak resistance. Default value: 100e6 [Ohm]
      ureset          : Reset potential. Default value: -70e-3[V]
      theta           : Firing threashold. Default value: -50e-3 [V]
    Returns:
      u               : membrane potential [V]
      ts              : spike times [s]
    """
    ts = []
    dt = t[1]-t[0]
    u = np.zeros(len(input_current))
    u[0]=urest
    for step in range(0,len(input_current)-1):
        if u[step] > theta:
            u[step] = theta/1.2 #purely for cosmetic reasons (to have the straight line)
            u[step+1] = ureset
            ts.append(t[step])
        else:
            u[step+1]=u[step] -dt/tau * (u[step]-urest) + r*dt/tau * input_current[step]
    return u, ts

t_max = 150e-3   # second
dt = .1e-3       # second
t_range = np.arange(0,t_max,dt)
input_current = syn_in(t_range)
u ,ts = simple_LIF(input_current, t_range)

theta = -50e-3 #For red dots
with plt.xkcd():
    fig, axs = plt.subplots(2)
    axs[0].plot(t_range,input_current)
    axs[0].set(title = "Input current", ylabel = "I [A]")
    axs[1].plot(t_range,u)
    axs[1].plot(ts,np.ones(len(ts))*theta/1.2,'r.',markersize=12)
    axs[1].set(title = "Membrane potential", ylabel = "u [V]", xlabel= "Time [S]")
    plt.show()

*Example output:*


<img alt='Solution hint' align='left' width=413 height=300 src=https://raw.githubusercontent.com/fblondiaux/LGBIO2072-2020/master/Images/TP1-Ex3.png>

---
# Refractory period

The absolute refractory period is a time interval in the order of a few milliseconds during which synaptic input will not lead to a 2nd spike, no matter how strong. This effect is due to the biophysics of the neuron membrane channels, and you can read more about absolute and relative refractory period [here](https://content.byui.edu/file/a236934c-3c60-4fe9-90aa-d343b3e3a640/1/module5/readings/refractory_periods.html) and [here](https://en.wikipedia.org/wiki/Refractory_period_(physiology)).

![refractory period plot](https://github.com/mpbrigham/colaboratory-figures/raw/master/nma/python-for-nma/refractory_period.png)

### Exercise 4 
Add the refractory period to your model

In [ ]:

def LIF(input_current,t,  tau= 20e-3 , urest= -60e-3 , r= 100e6, ureset = -70e-3 \
               ,theta = -50e-3, t_ref = 10e-3 ): 
    """
    Authors : Antoine de Comite and Florence Blondiaux
    Simulate the LIF model in response to an external input current

    Args:
      input_current   : Vector containing the input current [A].
      t               : Vector containing the discretized time points [s]
      tau             : Membrane time constant. Default value: 20e-3 [s]
      urest           : Leak potential. Default value: -60e-3 [V]
      r               : Leak resistance. Default value: 100e6 [Ohm]
      ureset          : Reset potential. Default value: -70e-3[V]
      theta           : Firing threashold. Default value: -50e-3 [V]
      t_ref           : Refractory period. Default value: 10e-3 [S]
    Returns:
      u               : membrane potential [V]
      ts              : spike times [s]
    """
    u = np.zeros_like(input_current)
    ts = []
    dt = t[1]-t[0]
    S = np.zeros_like(input_current)
    u[0]=urest
    t_wait = 0
    for step in range(0,len(input_current)-1):
        if u[step] > theta:
            u[step]= theta / 1.2
            u[step+1] = ureset
            t_wait = t_ref/dt
            ts.append((step+1)*dt)
        elif t_wait >0:
            u[step +1] = ureset
            t_wait -=1
        else:
            u[step+1]=u[step] -dt/tau * (u[step]-urest) + r*dt/tau * input_current[step]
    return u, ts

t_max = 150e-3   # second
dt = .1e-3        # second
t_range = np.arange(0,t_max,dt)
input_current = syn_in(t_range)
u,ts = LIF(input_current, t_range)

with plt.xkcd():
    fig, axs = plt.subplots(2)
    axs[0].plot(t_range,input_current)
    axs[0].set(title = "Input current", ylabel = "I [A]")
    axs[1].plot(t_range,u)
    axs[1].plot(ts,np.ones(len(ts))*-50e-3/1.2,'r.',markersize=12)
    axs[1].set(title = "Membrane potential", ylabel = "u [V]", xlabel= "Time [S]")
    plt.show()

Click [here](https://github.com/fblondiaux/LGBIO2072-2020/blob/master/Solutions/TP1-Ex4.py) for solution.

*Example output:*


<img alt='Solution hint' align='left' width=413 height=300 src=https://raw.githubusercontent.com/fblondiaux/LGBIO2072-2020/master/Images/TP1-Ex4.png>

### Exercise 5
Explore different values for LIF parameters. *tref* is expressed in [ms], *R* in [MOhm], *tau_m* in [ms] and *theta_th* in [mV].

In [ ]:
my_layout.width = '450px'
@widgets.interact(
    tref=widgets.FloatSlider(10, min=0, max=30, step=2, layout=my_layout),
    R=widgets.FloatSlider(100, min=10, max=200, step=10, layout=my_layout),
    tau_m = widgets.FloatSlider(20, min=5, max=40, step=2.5, layout=my_layout),
    theta_th = widgets.FloatSlider(-50, min=-55, max=-45, step=1, layout=my_layout)
)

def diff_tref(tref = 10e-3, R = 100, tau_m= 20, theta_th = -50):
    
    t_max = 150e-3   # second
    dt = .1e-3        # second
    t_range = np.arange(0,t_max,dt)
    input_current = syn_in(t_range)
    u,ts = LIF(input_current, t_range,t_ref = tref*1e-3, r=R*1e6, tau = tau_m *1e-3,theta = theta_th*1e-3 )

    fig, axs = plt.subplots(2)
    axs[0].plot(t_range,input_current)
    axs[0].set(title = "Input current", ylabel = "I [A]")
    axs[1].plot(t_range,u)
    axs[1].plot(ts,np.ones(len(ts))*-40e-3,'r.',markersize=12)
    axs[1].set(title = "Membrane potential", ylabel = "u [V]", xlabel= "Time [S]")
    plt.show()

---
## Random synaptic input
From the perspective of neurons, synaptic input is random (or stochastic). We'll improve the synaptic input model by introducing random input current with statistical properties similar to the previous exercise:

\begin{align*}
\\
I(t)=I_{mean}\left(1+0.1\sqrt{\frac{t_{max}}{\Delta t}}\,\xi(t)\right)\qquad\text{with }\xi(t)\sim \mathcal{N}(0,\sigma)\\
\\
\end{align*}

where $\mathcal{N}(0,\sigma)$ is the [gaussian distribution](https://en.wikipedia.org/wiki/Normal_distribution).

Random synaptic input $I(t)$ results in random time course for $u(t)$.

### Exercise 6
Implement a function that generates the input current given above


In [ ]:
def rand_syn(t,sigma = 0.1, imean = 25e-11):
    """
    Function that produces random input current

    Args:
      t          : vector of discretized time points 
      imean      : current mean amplitude [A]
      sigma      : variance of the random input current

    Returns:
      sin_i       : Sinusoidal input current
    """
    dt = t[1]-t[0]
    rand_i = imean * (1 + 0.1 * (np.sqrt(np.max(t)/dt)) * np.random.normal(0,sigma,len(t)))
    return rand_i

t_max = 150e-3   # second
dt = .1e-3        # second
t_range = np.arange(0,t_max,dt)

with plt.xkcd():
    plt.plot(t_range,rand_syn(t_range))
    plt.title('Input current')
    plt.ylabel('I [A]')
    plt.xlabel('T [S]')
    plt.show()
    
    

*Example output:*


<img alt='Solution hint' align='left' width=413 height=300 src=https://raw.githubusercontent.com/fblondiaux/LGBIO2072-2020/master/Images/TP1-Ex6.png>

Once you have implemented the random input current, explore the effect different values of sigma have on the membrane potential

In [ ]:
# @title
# @markdown Make sure you execute this cell to enable the widget!
my_layout.width = '450px'
@widgets.interact(
    sigma=widgets.FloatSlider(0.8, min=0, max=5, step=0.2, layout=my_layout)

)

def diff_DC(sigma = 0.1):
    t_max = 150e-3   # second
    dt = 1e-3        # second
    t_range = np.arange(0,t_max,dt)
    input_current = rand_syn(t_range, sigma)
    u,ts = LIF(input_current, t_range)
    
    fig, axs = plt.subplots(2)
    axs[0].plot(t_range,input_current)
    axs[0].set(title = "Input current", ylabel = "I [A]")
    axs[1].plot(t_range,u)
    axs[1].plot(ts,np.ones(len(ts))*-40e-3,'r.',markersize=12)
    axs[1].set(title = "Membrane potential", ylabel = "u [V]", xlabel= "Time [S]")
    plt.show()

---
## Bonus Section 1: Orenstein-Uhlenbeck Process

When a neuron receives spiking input, the synaptic current is Shot Noise -- which is a kind of colored noise and the spectrum of the noise determined by the synaptic kernel time constant. That is, a neuron is driven by **colored noise** and not GWN.

We can model colored noise using the Ohrenstein-Uhlenbeck process - filtered white noise. 

We next study if the input current is temporally correlated and is modeled as an Ornstein-Uhlenbeck process $\eta(t)$, i.e., low-pass filtered GWN with a time constant $\tau_{\eta}$: 

$$\tau_\eta \frac{d}{dt}\eta(t) = \mu-\eta(t) + \sigma_\eta\sqrt{2\tau_\eta}\xi(t).$$

**Hint:** An OU process as defined above has

$$E[\eta(t)]=\mu$$ 

and autocovariance 

$$[\eta(t)\eta(t+\tau)]=\sigma_\eta^2e^{-|t-\tau|/\tau_\eta},$$

which can be used to check your code.

In [ ]:
def my_OU(range_t, tau_ou, mu, sig, myseed=2072):
    """
    Function that produces Ornstein-Uhlenbeck input
    Args:
        tau_ou     : time constant [s]
        mu         : mean noise [A]
        sig        : noise amplitude [A]
        myseed     : random seed. int or boolean

    Returns:
        I_ou       : Ornstein-Uhlenbeck input current [A]
    """
    # set random seed
    if myseed:
      np.random.seed(seed=myseed)
    else:
      np.random.seed()
    # Retrieve simulation parameters
    dt=range_t[1]-range_t[0]
    Lt = range_t.size
    

    # Initialize
    noise = np.random.randn(Lt)
    I_ou = np.zeros(Lt)
    I_ou[0] = noise[0] * sig

    # generate OU
    for it in range(Lt-1):
        I_ou[it+1] = I_ou[it] + (dt / tau_ou) * (mu - I_ou[it]) + np.sqrt(2 * dt / tau_ou) * sig * noise[it + 1]

    return I_ou
t_max = 1000e-3   # second
dt = .1e-3        # second
range_t = np.arange(0,t_max,dt)
I_ou = my_OU(range_t=range_t,tau_ou=20e-3,mu=100e-12, sig=20e-12)
with plt.xkcd():
    plt.plot(range_t,I_ou)
    plt.title('Input current')
    plt.ylabel('I [A]')
    plt.xlabel('T [S]')
    plt.show()


*Example output:*


<img alt='Solution hint' align='left' width=413 height=300 src=https://raw.githubusercontent.com/fblondiaux/LGBIO2072-2020/master/Images/TP1-Ex7.png>

### Interactive Demo: LIF Explorer with OU input
In the following, we will check how a neuron responds to a noisy current that follows the statistics of an OU process. *tau_ou* is expressed in [ms] and *mu_ou* and *sigma_ou* in [pA] 

In [ ]:
my_layout.width = '450px'
@widgets.interact(
    tau_ou=widgets.FloatSlider(20.0, min=15, max=25,
                             step=2.5, layout=my_layout), 
    mu_ou=widgets.FloatSlider(100, min=70, max=170, step=10, layout=my_layout),
    sig_ou=widgets.FloatSlider(20, min=0, max=150,
                             step=10, layout=my_layout)

)


def LIF_with_OU(tau_ou=10 , mu_ou=25e-11, sig_ou=10e-11):
    t_max = 1000e-3   # second
    dt = .1e-3        # second
    t_range = np.arange(0,t_max,dt)

    I_ou = my_OU(t_range,tau_ou*1e-3,mu_ou * 1e-12, sig_ou*1e-12)
    u,stime = LIF(I_ou,t_range)

    plt.figure(figsize=(12, 4))
    plt.subplot(121)
    plt.plot(t_range, I_ou, 'b', lw=1.0)
    plt.xlabel('Time (ms)')
    plt.ylabel(r'$I_{\mathrm{OU}}$')
    plt.subplot(122)
    plt.plot(t_range, u)
    plt.xlabel('Time (s)')
    plt.ylabel('u (V)')
    plt.tight_layout()
    plt.show()

---
## Bonus Section 2: Generalized Integrate-and-Fire models

LIF model is not the only abstraction of real neurons. If you want to learn about more realistic types of neuronal models, watch a Bonus Video!

In [ ]:
# @title Video 3 (Bonus): Extensions to Integrate-and-Fire models
from ipywidgets import widgets

out2 = widgets.Output()
with out2:
  from IPython.display import IFrame
  class BiliVideo(IFrame):
      def __init__(self, id, page=1, width=400, height=300, **kwargs):
          self.id=id
          src = 'https://player.bilibili.com/player.html?bvid={0}&page={1}'.format(id, page)
          super(BiliVideo, self).__init__(src, width, height, **kwargs)

  video = BiliVideo(id="", width=730, height=410, fs=1)
  print('Video available at https://www.bilibili.com/video/{0}'.format(video.id))
  display(video)

out1 = widgets.Output()
with out1:
  from IPython.display import YouTubeVideo
  video = YouTubeVideo(id="G0b6wLhuQxE", width=730, height=410, fs=1, rel=0)
  print('Video available at https://youtube.com/watch?v=' + video.id)
  display(video)

out = widgets.Tab([out1, out2])
out.set_title(0, 'Youtube')
out.set_title(1, 'Bilibili')

display(out)